# Tarea 1
Dado el corpus proporcionado, obtener la curva de Zipf de las palabras en
escala logaritmica.
Pasos:

1. Limpiar el corpus: eliminar signos de puntuacion, de interrogacion, admiracion y elementos no lexicos.

2. Aplicar un algoritmo de Stemming a los tokens limpios.
3. Obtener las frecuencias de los tipos en el corpus.
4. Ordenar por el rango estadistico de mayor a menor.
5. Graficar el diagrama de dispersion rango-frecuencia en escala logarıtmica.

In [4]:
# Importamos las librerias

# Usamos os para cargar los libros
from os import listdir,getcwd
from os.path import isfile, join
import re


## Importamos el corpus
Lo primero que hacemos es importar el corpus al notebook para que podamos utilizarlos.

In [85]:

# Obtenemos el path del folder donde se almacenan los corpus
folder_path = (getcwd() + r"\CorpusDocs")

# Los almacenamos en una lista donde se almacenan los nombres de los archivos.
# Esto es en caso de que usemos todos los corpus.
corpus_name_list = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]

def loadAllCorpus():
    """
    Esta funcion carga todos los corpus que estan en el folder Corpus Docs.
    """
    corpis = ''
    for file in corpus_name_list:
        with open("./CorpusDocs/" + file, 'r', encoding="utf8") as f:
            corpis += f.read()
    return corpis

def loadCorpus(corpus_name):
    """
    Esta funcion nos sirve para cargar un corpus especifico
    """
    with open("./CorpusDocs/" + corpus_name, 'r', encoding="utf8") as f:
        corpus = f.read()
    return corpus
        
def identifyWords(text):
    """
    Esta función divide las palabras por espacio y limpia las palabras de los caracteres especiales
    """
    vocales = {
        'á': 'a',
        'é': 'e',
        'í': 'i',
        'ó': 'o',
        'ú': 'u'
        }
    texto2 = ''
    for val in text.lower():
        if(val == '(' or val ==')' or val == '\\' or val =='{' or val =='}'  or val == '?' or val =='=' or val == ',' 
            or val == '_' or val == '-' or val == '.' or val =='[' or val ==']' or val ==':' or val =='0' or val == '1' 
            or val == '2' or val == '3' or val == '4' or val =='5' or val =='6' or val =='7' or val =='8' or val =='9' 
            or val =='α' or val =='/' or val =='ρ' or val =='^' or val == 'θ' or val == '+' or val == '-' or val == '↑'
            or val == '·' or val == '"'):
                val = ' '
        else:
            if(val == 'á' or val == 'é' or val == 'í' or val == 'ó' or val == 'ú'):
                val = vocales[val]
        texto2 = texto2 + val
    base = texto2.split()
    return base

In [76]:
texts = loadAllCorpus()
text = loadCorpus('acustica (2).txt')

In [87]:
base = identifyWords(text)
print(base)

['energia', 'sonora', 'la', 'energia', 'sonora', 'o', 'energia', 'acustica', 'es', 'la', 'energia', 'que', 'transmiten', 'o', 'transportan', 'las', 'ondas', 'sonoras', 'procede', 'de', 'la', 'energia', 'vibracional', 'del', 'foco', 'sonoro', 'y', 'se', 'propaga', 'a', 'las', 'particulas', 'del', 'medio', 'que', 'atraviesan', 'en', 'forma', 'de', 'energia', 'cinetica', 'movimiento', 'de', 'las', 'particulas', 'y', 'de', 'energia', 'potencial', 'cambios', 'de', 'presion', 'producidos', 'en', 'dicho', 'medio', 'o', 'presion', 'sonora', 'al', 'irse', 'propagando', 'el', 'sonido', 'a', 'traves', 'del', 'medio', 'la', 'energia', 'se', 'transmite', 'a', 'la', 'velocidad', 'de', 'la', 'onda', 'pero', 'una', 'parte', 'de', 'la', 'energia', 'sonora', 'se', 'disipa', 'en', 'forma', 'de', 'energia', 'termica', 'la', 'energia', 'acustica', 'suele', 'tener', 'valores', 'absolutos', 'bajos', 'y', 'su', 'unidad', 'de', 'medida', 'es', 'el', 'joule', 'j', 'aunque', 'puede', 'calcularse', 'a', 'partir',